In [1]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


     |████████████████████████████████| 4.9 MB 30.3 MB/s 
     |████████████████████████████████| 462 kB 61.7 MB/s 


In [2]:
### GET BERT Model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [36]:
## GET dataset
import pandas as pd
df_train = pd.read_csv('Corona_NLP_train.csv' , encoding = "ISO-8859-1", index_col = False)
df_train = df_train.drop(columns = ['UserName' , 'ScreenName' , 'Location' , 'TweetAt'])
sentiment_list = ['Extremely Negative','Negative','Neutral','Positive','Extremely Positive'] 
df_train['Sentiment'] = df_train['Sentiment'].apply(lambda x: sentiment_list.index(x))
df_train.head(5)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2
1,advice Talk to your neighbours family to excha...,3
2,Coronavirus Australia: Woolworths to give elde...,3
3,My food stock is not the only one which is emp...,3
4,"Me, ready to go at supermarket during the #COV...",0


In [37]:
df_test = pd.read_csv('Corona_NLP_test.csv' , encoding = "ISO-8859-1", index_col = False)
df_test = df_test.drop(columns = ['UserName' , 'ScreenName' , 'Location' , 'TweetAt'])
df_test['Sentiment'] = df_test['Sentiment'].apply(lambda x: sentiment_list.index(x))
df_test.head(5)

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,3
2,Find out how you can protect yourself and love...,4
3,#Panic buying hits #NewYork City as anxious sh...,1
4,#toiletpaper #dunnypaper #coronavirus #coronav...,2


In [38]:
##One hot encoding
from sklearn.preprocessing import OneHotEncoder
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df_train = pd.DataFrame(enc.fit_transform(df_train[['Sentiment']]).toarray())
# merge with main df bridge_df on key values
df_train = df_train.join(enc_df_train)
pd.set_option('display.max_columns', None)
print(df_train.head(5))


                                       OriginalTweet  Sentiment    0    1  \
0  @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...          2  0.0  0.0   
1  advice Talk to your neighbours family to excha...          3  0.0  0.0   
2  Coronavirus Australia: Woolworths to give elde...          3  0.0  0.0   
3  My food stock is not the only one which is emp...          3  0.0  0.0   
4  Me, ready to go at supermarket during the #COV...          0  1.0  0.0   

     2    3    4  
0  1.0  0.0  0.0  
1  0.0  1.0  0.0  
2  0.0  1.0  0.0  
3  0.0  1.0  0.0  
4  0.0  0.0  0.0  


In [39]:
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df_test = pd.DataFrame(enc.fit_transform(df_test[['Sentiment']]).toarray())
# merge with main df bridge_df on key values
df_test = df_test.join(enc_df_test)
print(df_test.head(5))


                                       OriginalTweet  Sentiment    0    1  \
0  TRENDING: New Yorkers encounter empty supermar...          0  1.0  0.0   
1  When I couldn't find hand sanitizer at Fred Me...          3  0.0  0.0   
2  Find out how you can protect yourself and love...          4  0.0  0.0   
3  #Panic buying hits #NewYork City as anxious sh...          1  0.0  1.0   
4  #toiletpaper #dunnypaper #coronavirus #coronav...          2  0.0  0.0   

     2    3    4  
0  0.0  0.0  0.0  
1  0.0  1.0  0.0  
2  0.0  0.0  1.0  
3  0.0  0.0  0.0  
4  1.0  0.0  0.0  


In [40]:
## train  test split
from sklearn.model_selection import train_test_split
X_train ,X_test, y_train,y_test = train_test_split(df_train['OriginalTweet'],df_train[[0,1,2,3,4]] ,stratify = df_train[[0,1,2,3,4]],  test_size = 0.1)

In [55]:
## build a model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
output = bert_encoder(preprocessed_text)

# Neural network layers
l1 = tf.keras.layers.Dense(64, activation='relu', name="l1")(output['pooled_output'])
l2 = tf.keras.layers.Dropout(0.1, name="dropout")(l1)
l3 = tf.keras.layers.Dense(5, activation='softmax', name="l3")(l2)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l3])

In [45]:
print(X_train[:10])

9129     Police officers hand out rolls of toilet paper...
20610    Last night I had a dream that the grocery stor...
23587    UV lights for hospital and supermarket entranc...
31545    COVID 19 Why hoarding supplies is human nature...
15849    @loblawco @sobeys @METRO_RICHELIEU as Canadian...
14862    Anti Bacterial disinfectant spray\r\r\n\r\r\nC...
12890     In Phase One we wanted to focus our attention...
12986    Strong thread on changing consumer behavior in...
16055    Wht Hse's Navarro: In charge of "building up r...
3500     i am extremely worried that govts will use #co...
Name: OriginalTweet, dtype: object


In [56]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=25)


Epoch 1/25
1158/1158 [==============================] - 370s 318ms/step - loss: 1.5636 - accuracy: 0.2792
Epoch 2/25
1158/1158 [==============================] - 372s 322ms/step - loss: 1.5335 - accuracy: 0.2958
Epoch 3/25
1158/1158 [==============================] - 373s 322ms/step - loss: 1.5150 - accuracy: 0.3068
Epoch 4/25
 420/1158 [=========>....................] - ETA: 3:57 - loss: 1.5087 - accuracy: 0.3170

In [ ]:
model.evaluate(X_test, y_test)
